In [48]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

# categroy - (1)
root_dir = "./image/"
categories = ["meat", "noodle", "cat", "flower"]
np_classes = len(categories)
image_size = 50

# read images in folder - (2)
X = []
Y = []

for idx, folder in enumerate(categories):
    image_dir = root_dir + "/" + folder
    files = glob.glob(image_dir + "/*.jpg")
    print("---", folder, "처리중")
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_size, image_size))
        data = np.asarray(img)
        X.append(data)
        Y.append(idx)

X = np.array(X)
Y = np.array(Y)

# split the testing data and the training data - (3)
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y)

xy = (X_train, X_test, Y_train, Y_test)
np.save("./image/check_test.npy", xy)
print("end", len(Y))


--- meat 처리중
--- noodle 처리중
--- cat 처리중
--- flower 처리중
end 2001


In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils

import numpy as np

root_dir = "./image/"
categories = ["meat", "noodle", "cat", "flower"]
nb_classes = len(categories)
image_size = 50

In [6]:
# download data
def main():
    X_train, X_test, Y_train, Y_test = np.load("./image/check_test.npy")
    X_train = X_train.astype("float") / 256
    X_test = X_test.astype("float") / 256
    Y_train = np_utils.to_categorical(Y_train, nb_classes)
    Y_test = np_utils.to_categorical(Y_test, nb_classes)
    
    model = model_train(X_train, Y_train)
    model_eval(model, X_test, Y_test)

In [7]:
# build model
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='binary_crossentropy', 
                  optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [8]:
# model training
def model_train(X, Y):
    model = build_model(X.shape[1:])
    model.fit(X, Y, batch_size=32, nb_epoch=30)
    
    hdf5_file = "./image/model.hdf5"
    model.save_weights(hdf5_file)
    return model

In [9]:
# evaluate model
def model_eval(model, X, Y):
    score = model.evaluate(X, Y)
    print("loss =", score[0])
    print("accuracy =", score[1])

In [10]:
if __name__ == "__main__":
    main()

C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(50, 50, 3..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  import sys
C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if __name__ == '__main__':
C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7157 - accuracy: 0.7347
Epoch 2/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5042 - accuracy: 0.7670
Epoch 3/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4281 - accuracy: 0.8093
Epoch 4/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3555 - accuracy: 0.8538
Epoch 5/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2943 - accuracy: 0.8757
Epoch 6/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2597 - accuracy: 0.8932
Epoch 7/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2314 - accuracy: 0.9055
Epoch 8/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1919 - accuracy: 0.9232
Epoch 9/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1720 - accuracy: 0.9342
Epoch 10/30
1500/1500 [==============================] - 8s 5ms/step - loss: 0.148

In [47]:
from PIL import Image
import os, glob


image_size = 50
categories = ["meat", "noodle", "cat", "flower"]


# read smaple image
X = []
files = []
sample = ["sample_cat.jpg", "sample_flower.jpg", "sample_meat.jpg", "sample_noodle.jpg"]

for fname in sample:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((image_size, image_size))
    in_data = np.asarray(img)
    X.append(in_data)
    files.append(fname)
X = np.array(X)


model = build_model(X.shape[1:])
model.load_weights("./image/model.hdf5")

html = ""
pre = model.predict(X)
for i, p in enumerate(pre):
    y = p.argmax()
    print("file name:", files[i])
    print("categories:", categories[y])
    html += """
        <h3>file name: {0}</h3>
        <div>
            <p><img src="{1}" width=300></p>
            <p>categories: {2}
        </div>
            """.format(os.path.basename(files[i]), files[i], categories[y])

html = "<html><body style='text-align:center;'>" + \
    "<style> p { margin:0; padding:0; } </style>" + \
    html + "</body></html>"

with open("result.html", "w") as f:
    f.write(html)

C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(50, 50, 3..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  import sys
C:\Users\kjv45\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if __name__ == '__main__':


file name: sample_cat.jpg
categories: flower
file name: sample_flower.jpg
categories: flower
file name: sample_meat.jpg
categories: meat
file name: sample_noodle.jpg
categories: noodle
